In [1]:
import os

## 1 generate camera positions

In [1]:
config_folder = './configs'
n_views = 120
is_test = True

cmd = f'python ./scripts/generate_camera_pose.py ' \
      f'--n_views {n_views} ' \
      f'--is_test {is_test} ' \
      f'--out_folder {config_folder}'
print(cmd)

python ./scripts/generate_camera_pose.py --n_views 120 --is_test True --out_folder ./configs


## 2 blender rendering command

In [3]:
scenes = [
    'd19b8b80-a9af-477a-bf3f-41cd20cf840b.json',    # no
    'd1992aca-b517-4d27-822a-5adb08fd20fe.json',    # no
    'ff48ced7-689e-4292-a199-47435e73e3fa.json',    # no
    'ffe76c93-7b20-424a-8d58-a4fa7c81d4e8.json',    # v4
    '00321452-517d-431b-a21b-f52750684910.json',
]

scene = scenes[3]

### 2.1 render human with environment

In [4]:
data_root = '/home/xietao/data/3d-front'

mode = 'run'      # ['run', 'debug']
scene_config = os.path.join(data_root, f'3D-FRONT/{scene}')
objects_folder = os.path.join(data_root, f'3D-FUTURE-model')
texture_folder = os.path.join(data_root, f'3D-FRONT-texture')
human = 'nathan'
frame = 1
human_frame_path = f'/home/xietao/data/synthetic_human/{human}/object/{frame:06d}.obj'
config_folder = f'./configs'
output_folder = f'./output/env_{scene[:8]}_{human}'

# blender render command
cmd = f"blenderproc {mode} human_env_blend_visualizer.py " \
      f"--front {scene_config} " \
      f"--future_folder {objects_folder} " \
      f"--front_3D_texture_path {texture_folder} " \
      f"--digital_human_path {human_frame_path} " \
      f"--digital_human_config {config_folder}/digital_{human}_configs.json " \
      f"--camera_param_path {config_folder}/camera_parameters.json " \
      f"--camera_poses_path {config_folder}/camera_positions " \
      f"--output_dir {output_folder}/hdf5"
print(cmd)
print()

# file format convertion command
cmd = f'python ./scripts/hd5tojpg.py ' \
      f'--raw_folder {output_folder}/hdf5 ' \
      f'--out_folder {output_folder}/images'
print(cmd)

blenderproc run human_env_blend_visualizer.py --front /home/xietao/data/3d-front/3D-FRONT/ffe76c93-7b20-424a-8d58-a4fa7c81d4e8.json --future_folder /home/xietao/data/3d-front/3D-FUTURE-model --front_3D_texture_path /home/xietao/data/3d-front/3D-FRONT-texture --digital_human_path /home/xietao/data/synthetic_human/nathan/object/000001.obj --digital_human_config ./configs/digital_nathan_configs.json --camera_param_path ./configs/camera_parameters.json --camera_poses_path ./configs/camera_positions --output_dir ./output/env_ffe76c93_nathan/hdf5

python ./scripts/hd5tojpg.py --raw_folder ./output/env_ffe76c93_nathan/hdf5 --out_folder ./output/env_ffe76c93_nathan/images


### 2.2 render human only

In [ ]:
human_data_root = '/home/xietao/data/synthetic_human'
camera_num = 10
mode = 'run'
target_human = 'nathan'
frames = range(0, 31)
config_folder = f'./configs'
output_folder = f'./output/only_{target_human}'

for frame in frames:
    # render all views of one single frame
    human_frame_path = f'{human_data_root}/{target_human}/object/{frame:06d}.obj'
    cmd = f"blenderproc {mode} human_only_renderer.py " \
          f"--digital_human_path {human_frame_path} " \
          f"--digital_human_config {config_folder}/digital_{target_human}_configs.json " \
          f"--camera_param_path {config_folder}/camera_parameters.json " \
          f"--camera_poses_path {config_folder}/camera_positions " \
          f"--output_dir {output_folder}/frame_{frame:02d}/hdf5"
    print(cmd)
    # os.system(cmd)

for frame in frames:
    # convert `.hdf5` to `.jpg`
    cmd = f'python ./scripts/hd5tojpg_plus_mask.py ' \
          f'--raw_folder {output_folder}/frame_{frame:02d}/hdf5 ' \
          f'--img_folder {output_folder}/frame_{frame:02d}/images ' \
          f'--msk_folder {output_folder}/frame_{frame:02d}/mask'
    print(cmd)
    # os.system(cmd)

for i in range(camera_num):
    cmd = f'mkdir -p ./data/synthetic_human/{target_human}/images/{i:02d}'
    print(cmd)
    cmd = f'mkdir -p ./data/synthetic_human/{target_human}/mask/{i:02d}'
    print(cmd)

for frame in frames:
    # rename and move
    for i in range(camera_num):
        # move image
        source_img_path = f'{output_folder}/frame_{frame:02d}/images/{i}.jpg'
        target_img_path = f'./data/synthetic_human/{target_human}/images/{i:02d}/{frame:06d}.jpg'
        cmd = f'cp {source_img_path} {target_img_path}'
        print(cmd)
        # move mask
        source_msk_path = f'{output_folder}/frame_{frame:02d}/mask/{i}.png'
        target_msk_path = f'./data/synthetic_human/{target_human}/mask/{i:02d}/{frame:06d}.png'
        cmd = f'cp {source_msk_path} {target_msk_path}'
        print(cmd)

### 2.3 render envirment only

In [6]:
data_root = '/home/xietao/data/3d-front'

mode = 'run'      # ['run', 'debug']
scene_config = os.path.join(data_root, f'3D-FRONT/{scene}')
objects_folder = os.path.join(data_root, f'3D-FUTURE-model')
texture_folder = os.path.join(data_root, f'3D-FRONT-texture')
config_folder = f'./configs'
output_folder = f'./data/3d-front/{scene[:8]}'

# blender render command
cmd = f"blenderproc {mode} env_only_renderer.py " \
      f"--front {scene_config} " \
      f"--future_folder {objects_folder} " \
      f"--front_3D_texture_path {texture_folder} " \
      f"--camera_param_path {config_folder}/camera_parameters.json " \
      f"--camera_poses_path {config_folder}/camera_positions " \
      f"--output_dir {output_folder}/hdf5"
print(cmd)
print()

# file format convertion command
cmd = f'python ./scripts/hd5tojpg.py ' \
      f'--raw_folder {output_folder}/hdf5 ' \
      f'--out_folder {output_folder}/images'
print(cmd)

blenderproc run env_only_renderer.py --front /home/xietao/data/3d-front/3D-FRONT/ffe76c93-7b20-424a-8d58-a4fa7c81d4e8.json --future_folder /home/xietao/data/3d-front/3D-FUTURE-model --front_3D_texture_path /home/xietao/data/3d-front/3D-FRONT-texture --camera_param_path ./configs/camera_parameters.json --camera_poses_path ./configs/camera_positions --output_dir ./data/3d-front/ffe76c93/hdf5

python ./scripts/hd5tojpg.py --raw_folder ./data/3d-front/ffe76c93/hdf5 --out_folder ./data/3d-front/ffe76c93/images


## 3 generate `easymocap` format camera parameters

In [7]:
human = 'nathan'
human_root = f'./data/synthetic_human/{human}'
param_root = f'./configs'

cmd = f'blenderproc run ./scripts/generate_camera_params.py ' \
      f'--camera_intri_path {param_root}/camera_parameters.json ' \
      f'--camera_extri_path {param_root}/camera_positions ' \
      f'--output_path {human_root}'
print(cmd)

blenderproc run ./scripts/generate_camera_params.py --camera_intri_path ./configs/camera_parameters.json --camera_extri_path ./configs/camera_positions --output_path ./data/synthetic_human/nathan


## 4 prepare data for `phdeform` lib

In [8]:
# cd the 3rd lib `easymocap` 
easymocap_path = './EasyMocap'
cmd = f'cd {easymocap_path}'
print(cmd)

# preprocess the images to extract the 2d keypoints
# https://chingswy.github.io/easymocap-public-doc/quickstart/keypoints.html#extract-keypoints
data_root = f'/home/xietao/code/blender/blender_digital_human/data/synthetic_human'
human = f'nathan'
cmd = f'python apps/preprocess/extract_keypoints.py {data_root}/{human} --mode mp-holistic'
print(cmd)

# motion capture using `smplh` model
work = 'smplh-3d'
cmd = f'python3 apps/demo/mocap.py {data_root}/{human} --mode {work} --ranges 0 31 1'
print(cmd)

# generate `motion.npz` used by `phdeform` base dataset
cmd = f'python3 ./scripts/generate_annots_motion.py --data_folder ./data/synthetic_human/{human}'
print(cmd)

cd ./EasyMocap
python apps/preprocess/extract_keypoints.py /home/xietao/code/blender/blender_digital_human/data/synthetic_human/nathan --mode mp-holistic
python3 apps/demo/mocap.py /home/xietao/code/blender/blender_digital_human/data/synthetic_human/nathan --mode smplh-3d --ranges 0 31 1
python3 ./scripts/generate_annots_motion.py --data_folder ./data/synthetic_human/nathan
